# Examen ETL: SPARK 02/02 - María Carrasco Meléndez

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de partidosLigaNBA.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

## Primera pregunta: Describe brevemente que diferencia el persists, cache y collect en spark. Explica brevemente casos en los que es interesante su aplicación

Persists: Set this RDD’s storage level to persist its values across operations after the first time it is computed. This can only be used to assign a new storage level if the RDD does not have a storage level set yet. If no storage level is specified defaults to (MEMORY_ONLY).

Cache: Persist this RDD with the default storage level (MEMORY_ONLY).

Collect: Return a list that contains all of the elements in this RDD. This method should only be used if the resulting array is expected to be small, as all the data is loaded into the driver’s memory.

Si vamos a trabajar mucho sobre un mismo RDD, es conveniente moverlo a memoria para evitar que Spark esté generándolo en cada acción que hagamos. Si vamos a trabajar mucho sobre un mismo RDD, es conveniente moverlo a memoria para evitar que Spark esté generándolo en cada acción que hagamos. En el caso de collect, este método es conveniente utilizarlo al final de cualquier trabajo ya que carga todos los datos a diferencia de ola fincion .take. De esta manera, ahorraremos memoria.

## Segunda pregunta: Explica brevemente los pasos a seguir para realizar la carga de un conjunto de datos (pasos que se siguieron en la práctica con datos de logs)

En primer lugar, tendremos que comprobar si se encuentran en el mismo directorio nuestra fuente de datos y nuestro notebook. Una vez que disponemos de ambos ubicados en una misma carpeta, ejecutaremos la función correspondiente para la lectura de los datos dependiendo de su formato si es csv, json o yaml. Cabe destacar que esta función dependerá de si nos encontramos usando pandas o pyspark.

La función es la siguiente: from pyspark import SparkContext; sc = SparkContext("local", "First App"). Tras haber ejecutado y acabado nuestro trabajo tendremos que ejecutar la siguiente función: sc.stop().

Al cargar nuestra fuente de datos la guardaremos en un objeto que creemos.

## Tercera Pregunta: Indica un tipo de problema que puede empeorar los datos. (pe. Que no exista un representante del CDO en todas las áreas de negocio), pon algún ejemplo específico (pe. Datos duplicados) y cómo lo tratarías con técnicas de data cleaning.

Un problema que puede empeorar los datos son los valores nulos o NAs. Son valores que no aportan información relevante en nuestra base de datos y que por tanto, no siempre, es conveniente eliminarlos para estudiar por ejemplo correlaciones entre ellos.
Mediante la siguiente función, procedemos a eliminar los NAs en pyspark:
join_Df1= Name.filter(Name.col(FirstName).isnotnull()).show()

Sin embargo, debemos tener en cuenta que estos Nan son valores que pueden contener información, por ejemplo en el caso del test el dato puede tener Na porque no ha llegado la información a la vez al train y al test. Es conveniente tratarlo como una categoría distinta teniendo presente el overfitting y como se ha mencionado anteriormente el train y el test.

## Cuarta tarea: Inicializar spark context y cargar los datos desde el fichero.

In [11]:
from pyspark import SparkContext

In [12]:
sc = SparkContext()

In [13]:
NBA = "./partidosLigaNBA.csv"
NBA = sc.textFile(NBA)

#### Parseado de datos y análisis descriptivo de los mismos.

In [14]:
NBA = NBA.filter(lambda x: "PTS" not in x).map(lambda x: x.split(":")) #eliminamos el header para poder trabajar con los datos 

In [15]:
NBA.take(1)

[['Tue, Oct 30, 2007',
  '"7',
  '30 pm"',
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96']]

In [16]:
#es necesario eliminar tambien los playoffs ya que sino podemos tener problemas para realizar los cálculos

NBA = NBA.filter(lambda x: "Playoffs" not in x)

## Quinta tarea: Media de la diferencia de puntos por año

En este apartado, se pretende calcular la diferencia entre los puntos de los visitantes y los puntos en casa y se le aplica la media a dicho resultado.

In [20]:
 # En primer lugar obtenemos el año
año = NBA.map(lambda x: x[0][-4:])

In [21]:
#obtenemos los puntos visita y puntos casa
puntos_visita = NBA.map(lambda x: int(x[4]))
puntos_casa = NBA.map(lambda x: int(x[6]))

In [23]:
#unimos los puntos del equipo visitante con el año
puntos_año_visitante= NBA.map(lambda x: (x[0][-4:], int(x[4])))
puntos_año_visitante_key = puntos_año_visitante.reduceByKey(lambda x, y: x + y)
puntos_año_visitante_key.take(5)

[('2014', 133100),
 ('2013', 128963),
 ('2017', 84102),
 ('2016', 136221),
 ('2011', 86293)]

In [24]:
#unimos los puntos del equipo de casa con el año
puntos_año_casa = NBA.map(lambda x: (x[0][-4:], int(x[6])))
puntos_año_casa_key = puntos_año_casa.reduceByKey(lambda x, y: x + y)
puntos_año_casa_key.take(5)

[('2014', 136567),
 ('2013', 132642),
 ('2017', 86682),
 ('2016', 140466),
 ('2011', 88998)]

In [30]:
#Hacemos la diferencia
dif_puntos = NBA.map(lambda x: (x[0][-4:], abs(int(x[4])-int(x[6]))))
dif_puntos_key = dif_puntos.reduceByKey(lambda x, y: x + y)
dif_puntos_key.take(5)

[('2014', 14547),
 ('2013', 14659),
 ('2017', 9172),
 ('2016', 15397),
 ('2011', 9435)]

In [29]:
# Calculamos la media
media = round(dif_puntos.map(lambda x: (x[0][-4:], abs(int(x[4])-int(x[6])))).mean(),2)
media


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 18, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2371, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 2371, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 317, in func
    return f(iterator)
  File "/usr/local/spark/python/pyspark/

## Sexta tarea: ¿Han jugado todos los equipos el mismo número de partidos? Si es que no, ¿a qué puede deberse?

Se cuentan los partidos que ha jugado cada equipo, tanto visitante como de casa, durante todas las temporadas.

In [31]:
#En primer lugar, obtenemos el número de partidos jugados por el equipo visitante
partidos_eq_visitante = NBA.map(lambda x: (x[3], 1))
partidos_eq_visitante_key = partidos_eq_visitante.reduceByKey(lambda x, y: x + y)
partidos_eq_visitante_key.take(5)

[('Oklahoma City Thunder', 408),
 ('Cleveland Cavaliers', 452),
 ('Minnesota Timberwolves', 402),
 ('Chicago Bulls', 437),
 ('San Antonio Spurs', 466)]

In [32]:
#En segundo lugar, obtenemos el número de partidos jugados por el equipo de casa
partidos_eq_casa = NBA.map(lambda x: (x[5], 1))
partidos_eq_casa_key = partidos_eq_casa.reduceByKey(lambda x, y: x + y)
partidos_eq_casa_key.take(5)

[('Oklahoma City Thunder', 410),
 ('Cleveland Cavaliers', 449),
 ('Minnesota Timberwolves', 402),
 ('Chicago Bulls', 436),
 ('Utah Jazz', 422)]

In [34]:
#Mediante un join obtenemos los partidos comunes a ambos equipos
comun_partidos = partidos_eq_visitante_key.join(partidos_eq_casa_key)
comun_partidos.take(5)

[('Oklahoma City Thunder', (408, 410)),
 ('Cleveland Cavaliers', (452, 449)),
 ('Minnesota Timberwolves', (402, 402)),
 ('Chicago Bulls', (437, 436)),
 ('San Antonio Spurs', (466, 467))]

In [35]:
#Sumamos el total de los partidos
total_partidos = comun_partidos.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1])))).sortBy(lambda x: -x[1])
total_partidos.collect()

[('San Antonio Spurs', 933),
 ('Boston Celtics', 930),
 ('Miami Heat', 917),
 ('Cleveland Cavaliers', 901),
 ('Los Angeles Lakers', 897),
 ('Atlanta Hawks', 896),
 ('Golden State Warriors', 885),
 ('Chicago Bulls', 873),
 ('Houston Rockets', 868),
 ('Indiana Pacers', 868),
 ('Memphis Grizzlies', 867),
 ('Dallas Mavericks', 867),
 ('Orlando Magic', 863),
 ('Los Angeles Clippers', 861),
 ('Portland Trail Blazers', 853),
 ('Toronto Raptors', 850),
 ('Denver Nuggets', 848),
 ('Utah Jazz', 846),
 ('Washington Wizards', 844),
 ('Philadelphia 76ers', 834),
 ('Detroit Pistons', 829),
 ('Milwaukee Bucks', 827),
 ('Phoenix Suns', 825),
 ('New York Knicks', 825),
 ('Oklahoma City Thunder', 818),
 ('Minnesota Timberwolves', 804),
 ('Sacramento Kings', 804),
 ('Charlotte Bobcats', 566),
 ('New Orleans Hornets', 499),
 ('Brooklyn Nets', 435),
 ('New Jersey Nets', 394),
 ('New Orleans Pelicans', 332),
 ('Charlotte Hornets', 253),
 ('Seattle SuperSonics', 82)]

## Séptima pregunta: ¿Cuantos partidos ha ganado en Enero Clevelant?

En primer lugar queremos de todos los partidos saber cuales son los que ha ganado el equipo Cleveland s, para eso tendremos que aplicar un filter. Sin embargo, tenemos tambien que prestar atencion a que solo necesitamos el ems de enero por lo que se aplicarían dos filter

In [36]:
NBA.map(lambda x: x[0][5:8]).distinct().collect() 
#Necesitamos saber cuantos caracteres tiene enero

['May', 'Apr', 'Jan', 'Nov', 'Jun', 'Mar', 'Oct', 'Dec', 'Feb']

In [37]:
enero = NBA.filter(lambda x: x[0][5:8] == 'Jan') 
enero.take(1)

[['Wed, Jan 2, 2008',
  '"7',
  '30 pm"',
  'Houston Rockets',
  '93',
  'Boston Celtics',
  '97']]

In [38]:
NBA.map(lambda x: x[3]).distinct().collect() #buscamos el nombre exacto de nuestro equipo

['Oklahoma City Thunder',
 'Cleveland Cavaliers',
 'Minnesota Timberwolves',
 'Chicago Bulls',
 'San Antonio Spurs',
 'Memphis Grizzlies',
 'New Jersey Nets',
 'Miami Heat',
 'Brooklyn Nets',
 'Utah Jazz',
 'Orlando Magic',
 'Houston Rockets',
 'Indiana Pacers',
 'Seattle SuperSonics',
 'Washington Wizards',
 'Phoenix Suns',
 'Charlotte Hornets',
 'Sacramento Kings',
 'Toronto Raptors',
 'Portland Trail Blazers',
 'New Orleans Hornets',
 'Philadelphia 76ers',
 'Charlotte Bobcats',
 'Detroit Pistons',
 'Golden State Warriors',
 'Milwaukee Bucks',
 'New York Knicks',
 'Boston Celtics',
 'Atlanta Hawks',
 'Los Angeles Clippers',
 'Los Angeles Lakers',
 'New Orleans Pelicans',
 'Denver Nuggets',
 'Dallas Mavericks']

In [41]:
#Filtramos los partidos
cleveland_visita = enero.filter(lambda x: x[3] == 'Cleveland Cavaliers')  
cleveland_casa = enero.filter(lambda x: x[5] == 'Cleveland Cavaliers')  

In [42]:
#Sacamos los partidos ganados
ganados_visita = cleveland_visita.filter(lambda x: int(x[4]) > int(x[6]))
ganados_casa = cleveland_casa.filter(lambda x: int(x[6]) > int(x[4]))

In [44]:
ganados_total = ganados_visita + ganados_casa
ganados_total

UnionRDD[69] at union at NativeMethodAccessorImpl.java:-2

## Octava pregunta: ¿Los Warrios son mejores fuera de casa o en casa?

En este caso solo necesitamos saber si los warriors son mejores fuera de casa o en casa.

In [45]:
warriors_partidos_visit = NBA.filter(lambda x: x[3] == 'Golden State Warriors')
warriors_partidos_visit.take(1)

[['Fri, Nov 2, 2007',
  '"7',
  '30 pm"',
  'Golden State Warriors',
  '114',
  'Los Angeles Clippers',
  '120']]

In [47]:
warriors_partidos_casa = NBA.filter(lambda x: x[5] == 'Golden State Warriors')
warriors_partidos_casa.take(1)

[['Tue, Oct 30, 2007',
  '"7',
  '30 pm"',
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96']]

In [46]:
ganados_warriors_visit = warriors_partidos_visit.filter(lambda x: int(x[4]) > int(x[6]))
ganados_warriors_visit.count()

215

In [48]:
ganados_warriors_casa = warriors_partidos_casa.filter(lambda x: int(x[6]) > int(x[4]))
ganados_warriors_casa.count()

308

Como podemos observar los warriors han ganado más partidos en casa que fuera

## Novena pregunta: Equipo que ha quedado primerio en victorias más temporadas. (si es que hay alguno que más)

##  Décima pregunta: Escribe la expresión regular correcta que sólo macheen los teléfonos y el correo del siguiente texto.

Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com

In [10]:
sc.stop()